# Setup

## Dependencies

In [ ]:
# Standard libraries imports
import os
import shutil
import random
from pathlib import Path
import sys

# Third-party libraries imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from PIL import Image
from huggingface_hub import hf_hub_download

# Local imports
from src.gradcam.gradcam_utils import pre_gradcam, gradcam, post_gradcam, gradcam_plus_plus
from src.gradcam.model_loader import load_full_model

/home/mtoan65/AI/BioMedAI/Mammo_2025_Proj/mammoSGM/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# GradCAM

In [2]:
def load_data_bbx3(data_folder: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Load and process dataset with bounding box annotations from metadata CSV.

    This function reads metadata containing bounding box annotations, processes
    them into structured formats, and splits the data into training and test sets.
    It automatically tries to load 'metadata2.csv' first, falling back to 'metadata.csv'
    if not found.

    Parameters
    ----------
    data_folder : str
        Path to the folder containing the metadata CSV file.
        The folder should contain either 'metadata2.csv' or 'metadata.csv'.

    Returns
    -------
    train_df : pandas.DataFrame
        Training dataset with the following columns:

        - image_id : str or int
            Unique identifier for each image.
        - x, y, width, height : numeric
            Individual bounding box coordinates and dimensions.
        - split : str
            Dataset split identifier (value: "train").
        - bbx : list of numeric
            Single bounding box as [x, y, width, height].
        - bbx_list : list of list
            All bounding boxes for the same image_id, aggregated as
            [[x1, y1, w1, h1], [x2, y2, w2, h2], ...].
        - Additional columns from the original CSV as available.

    test_df : pandas.DataFrame
        Test dataset with the same column structure as train_df, but filtered
        for split == "test".

    Notes
    -----
    - The function prioritizes 'metadata2.csv' over 'metadata.csv'.
    - Each row in the original CSV represents one bounding box annotation.
    - The 'bbx' column contains individual box coordinates as a list.
    - The 'bbx_list' column aggregates all boxes belonging to the same image_id,
      enabling easy access to multi-object annotations.
    - Images with multiple objects will have multiple rows with the same image_id
      but different bounding box coordinates.
    - The split column must contain 'train' and 'test' values to partition the data.

    Examples
    --------
    >>> train_df, test_df = load_data_bbx3('data/annotations/')
    >>> print(f"Train samples: {len(train_df)}, Test samples: {len(test_df)}")
    Train samples: 1250, Test samples: 350

    >>> # Access bounding boxes for a specific image
    >>> image_boxes = train_df[train_df['image_id'] == 'img_001']['bbx_list'].iloc[0]
    >>> print(f"Image has {len(image_boxes)} objects")
    Image has 3 objects

    >>> # Each bounding box format
    >>> single_box = train_df.iloc[0]['bbx']
    >>> print(f"Bounding box: x={single_box[0]}, y={single_box[1]}, "
    ...       f"w={single_box[2]}, h={single_box[3]}")
    Bounding box: x=150, y=200, w=100, h=120

    >>> # Get all boxes for visualization
    >>> for idx, row in train_df.iterrows():
    ...     image_id = row['image_id']
    ...     all_boxes = row['bbx_list']  # List of all boxes for this image
    ...     # Draw boxes using post_gradcam or other visualization tools
    """
    metadata_path = os.path.join(data_folder, "metadata2.csv")
    if not os.path.exists(metadata_path):
        metadata_path = os.path.join(data_folder, "metadata.csv")

    df = pd.read_csv(metadata_path)

    # Create bbx column from x, y, width, height columns
    df["bbx"] = df[["x", "y", "width", "height"]].apply(
        lambda row: [row["x"], row["y"], row["width"], row["height"]], axis=1
    )

    # Group all bounding boxes by image_id into a list
    bbx_grouped = df.groupby("image_id")["bbx"].apply(list).reset_index().rename(columns={"bbx": "bbx_list"})
    df = df.merge(bbx_grouped, on="image_id", how="left")

    # Split into train and test sets
    train_df = df[df["split"] == "train"]
    test_df = df[df["split"] == "test"]

    return train_df, test_df

## Standard

In [ ]:
# thử nghiệm với ảnh thứ 21
dataset_folder = '/content/dataset/SGM1000bbx'
train_df, test_df = load_data_bbx3(dataset_folder)
test_df.iloc[21]

In [ ]:
# Example usage
dataset_folder = '/content/dataset/SGM1000bbx'
train_df, test_df = load_data_bbx3(dataset_folder)
pretrained_model_path = '/content/weight/mammoSGM/SGM1000bbx/SGM1000bbx_448x448_based_resnet50_8188_full.pth'

model_tuple = load_full_model(pretrained_model_path)

image_path = os.path.join(dataset_folder, test_df.iloc[21]['link'])
bbx_list = test_df.iloc[21]['bbx_list']
gt = test_df.iloc[21]['cancer']

model, input_tensor, img, target_layer, class_idx, pred_class, prob_class = pre_gradcam(model_tuple, image_path, target_layer=None, class_idx=None)
gradcam_map = gradcam(model, input_tensor, target_layer, class_idx)
#gradcam_map = gradcam_plus_plus(model, input_tensor, target_layer, class_idx)
post_gradcam(gradcam_map, img, bbx_list=bbx_list, option=5, blend_alpha=0.5, pred=pred_class, prob=prob_class, gt_label=gt)


In [ ]:
# Example usage
dataset_folder = '/content/dataset/SGM1000bbx'
train_df, test_df = load_data_bbx3(dataset_folder)
pretrained_model_path = '/content/weight/mammoSGM/Optimamv3/Optimamv3_448x448_based_convnextv2_tiny_9334_full.pth'

image_path = os.path.join(dataset_folder, test_df.iloc[41]['link'])
bbx_list = test_df.iloc[41]['bbx_list']
gt = test_df.iloc[41]['cancer']

model, input_tensor, img, target_layer, class_idx, pred_class, prob_class = pre_gradcam(model_tuple, image_path, target_layer=None, class_idx=None)
gradcam_map = gradcam(model, input_tensor, target_layer, class_idx)
post_gradcam(gradcam_map, img, bbx_list=bbx_list, option=5, blend_alpha=0.5, pred=pred_class, prob=prob_class, gt_label=gt)


## MIL